In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from nltk import word_tokenize
import pickle

In [2]:
embeddings_idx={}
f = open('glove.840B.300d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_idx[word] = coefs
    except ValueError:
        pass
f.close()

2196017it [03:30, 10454.21it/s]


In [3]:
type(embeddings_idx)

dict

###### load the data

In [4]:
trn = pd.read_csv('../preprocessed_data/trn_title.csv', delimiter = ',', names=['title','label'])
val = pd.read_csv('../preprocessed_data/val_title.csv', delimiter = ',', names=['title','label'])
tst = pd.read_csv('../preprocessed_data/tst_title.csv', delimiter = ',', names=['title','label'])

In [5]:
print(trn.shape)
print(val.shape)
print(tst.shape)

(30986, 2)
(8853, 2)
(4428, 2)


In [6]:
print(trn.values[10][0])

Trump Is LITERALLY Polling At <num> With Black People IMAGE VIDEO


In [7]:
def tokenize(sentence):
    # tokenize sentence
    return sentence.strip().split()

In [8]:
print(tokenize(trn.values[10][0]))

['Trump', 'Is', 'LITERALLY', 'Polling', 'At', '<num>', 'With', 'Black', 'People', 'IMAGE', 'VIDEO']


In [9]:
def sentenceToVector(s):
    # convert sentence to glove vector
    s = tokenize(s)
    M = []
    for word in s:
        try:
            M.append(embeddings_idx[word])
        except:
            M.append(embeddings_idx['unk'])
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
#     return v/np.sqrt((v ** 2).sum())
    return v

In [10]:
print(sentenceToVector(trn.values[10][0]).shape)

(300,)


In [11]:
# create all embeddings
# fit embeddings to labels
trn_embeddings = [sentenceToVector(x[0]) for x in tqdm(trn.values)]
val_embeddings = [sentenceToVector(x[0]) for x in tqdm(val.values)]
tst_embeddings = [sentenceToVector(x[0]) for x in tqdm(tst.values)]

f = open("../preprocessed_embeddings/GloVe_trn_title.pkl", "wb")
pickle.dump(trn_embeddings, f)
f.close()

f = open("../preprocessed_embeddings/GloVe_tst_title.pkl", "wb")
pickle.dump(tst_embeddings, f)
f.close()

100%|██████████| 4428/4428 [00:00<00:00, 44055.68it/s]
